<hr style="border:0.2px solid black"> </hr>

<figure>
  <IMG SRC="img/ntnu-norwegian-university-of-science-and-technology-vector-logo.png" WIDTH=250 ALIGN="right">
</figure>

**<ins>Course:</ins>** TVM4174 - Hydroinformatics for Smart Water Systems

# <ins>Example 4d:</ins> Simple Examples that EPANET can't do
    
*Developed by David B. Steffelbauer*

<hr style="border:0.2px solid black"> </hr>
    
## Monte Carlo Analysis

In [ ]:
import os
import wntr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
inp_file = 'data/L-TOWN_AreaC.inp'

wn = wntr.network.WaterNetworkModel(inp_file)
wntr.graphics.plot_network(wn)

In [ ]:
sim = wntr.sim.EpanetSimulator(wn)
res = sim.run_sim()
orig_pressure = res.node['pressure'].squeeze()

In [ ]:
# Perform Monte Carlo simulation with varying roughness parameters

mcsteps = 1000
mc_pressures = dict()

for ii in tqdm(range(mcsteps)):

    wn = wntr.network.WaterNetworkModel(inp_file)
    sim = wntr.sim.EpanetSimulator(wn)

    for pipe_name in wn.pipe_name_list:
        pipe = wn.get_link(pipe_name)
        r = pipe.roughness
        pipe.roughness = r + r*0.2*np.random.randn()

    res = sim.run_sim()
    mc_pressures[ii] = res.node['pressure'].squeeze()


mc_pressures = pd.DataFrame(mc_pressures)

In [ ]:
nodename = 'n389'

mc_pressures.loc[nodename].hist(bins=30)
plt.xlabel('pressure (m)')
plt.ylabel('occurences (-)')
plt.title(f'MCA for Node {nodename}')
plt.show()

In [ ]:
rmse = np.sqrt((mc_pressures.sub(orig_pressure, axis=0) ** 2).mean(axis=1))

wntr.graphics.plot_network(wn, node_attribute=rmse, node_colorbar_label='RMSE')
plt.show()